In [1]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import os

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

This environment uses conda install --file requirements.txt to install libraries

In [3]:
import time
import datetime
import csv
import logging
from dotenv import load_dotenv

The following code demonstrate how easy it is to use a remote webdriver. Here we specify that we want to use a headless Chrome browser (browser without visual display) for our driver, which is hosted on port 4444 of our hub service. 

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Remote(command_executor='http://hub:4444/wd/hub',
                         desired_capabilities=DesiredCapabilities.CHROME)

In [5]:
# simle test to get Google title
driver.get('https://www.google.com')
print(driver.title)

Google


## SETTING

In [6]:
load_dotenv()
LINKEDIN_USERNAME = os.getenv('LINKEDIN_USERNAME')
LINKEDIN_PASSWORD = os.getenv('LINKEDIN_PASSWORD')

class GEO_ID:
    SG = "102454443"
    MY = "106808692"
    
location = "Singapore"

In [7]:
# wait time for events in seconds
page_wait = 30
click_wait = 5
async_wait = 5

# when retrying, number of attempts
attempts = 3

In [8]:
print(LINKEDIN_USERNAME, ": ", LINKEDIN_PASSWORD)

xitonchong@gmail.com :  DD58sqUR5ifXcDq


In [9]:
url_login = "https://www.linkedin.com/"

In [10]:
search_posted = "r86400" # filter for the past 24 hours
search_posted = "r2592000" # filter for the past month
search_remote = "true"
search_location = "Singapore"
search_keywords = ['Data%20Scientist', 'Data Analyst', 'Data Engineer']

search_page = 0  # start on page 1
search_count = 1 # initiate search count until looks on page

### LOAD UTILS FILE

In [11]:
from utils import *

logging = create_logfile()
date = datetime.date.today().strftime('%d-%b-%y')
file = f"output/{date}_{search_location}.csv"
create_file(file, logging)

## LOGIN

In [12]:
logging.info(f"Logging in to LinkedIn as {LINKEDIN_USERNAME}...")

driver.get(url_login)
driver.find_element_by_id("session_key").send_keys(LINKEDIN_USERNAME)
driver.find_element_by_id("session_password").send_keys(LINKEDIN_PASSWORD)
driver.find_element_by_xpath("//button[@class='sign-in-form__submit-button']").click()

try: 
    driver.find_element_by_xpath("//button[@class='primary-action-new']").click()
except:
    print('failed')
    pass
logging.info("Log in complete. Scraping data...")

  

failed


In [13]:
## page search

In [14]:
url_search = f"https://www.linkedin.com/jobs/search/?f_TPR={search_posted}&geoId={GEO_ID.SG}&keywords={search_keywords[0]}&location={search_location}&start={search_page}"



In [15]:
print(url_search)

https://www.linkedin.com/jobs/search/?f_TPR=r2592000&geoId=102454443&keywords=Data%20Scientist&location=Singapore&start=0


In [16]:
driver.get(url_search)

#time.sleep(10) # add sleep so don't get caught

In [17]:
print(driver.title)

(1) Data Scientist Jobs in Singapore | LinkedIn


In [18]:
# find the number of results
print(driver.current_url)
search_count = driver.find_element_by_xpath("//small[@class='display-flex t-12 t-black--light t-normal']").text
search_count = int(search_count.split(' ')[0].replace(',', ''))  # get number before space & remove comma (ex. "1,245 results")
#logging.info(f"Loading page {round(search_page/25) + 1} of {round(search_count/25)} for {search_keyword}'s {search_count} results...")

https://www.linkedin.com/jobs/search/?f_TPR=r2592000&geoId=102454443&keywords=Data%20Scientist&location=Singapore&start=0


In [19]:
print(search_count)

2478


In [20]:
search_count = driver.find_element_by_xpath("//small[@class='display-flex t-12 t-black--light t-normal']")

In [21]:
search_count = driver.find_element_by_xpath("//small[@class='display-flex t-12 t-black--light t-normal']").text

In [22]:
try:
    search_results = driver.find_element_by_xpath("//ul[@class='jobs-search-results__list list-style-none']").find_elements_by_tag_name("li")
    result_ids = [result.get_attribute('id') for result in search_results if result.get_attribute('id') != '']
except:
    print('failed')
    time.sleep(click_wait) # wait a few attempts, if not throw an exception and then skip to next page

In [23]:
# get all the job_id's for xpath for current page to click each element


for attempt in range(attempts):
    try:
        search_results = driver.find_element_by_xpath("//ul[@class='jobs-search-results__list list-style-none']").find_elements_by_tag_name("li")
        result_ids = [result.get_attribute('id') for result in search_results if result.get_attribute('id') != '']
        break
    except:
        print('waiting')
        time.sleep(click_wait) # wait a few attempts, if not throw an exception and then skip to next page

In [24]:
print(result_ids)

['ember219', 'ember231', 'ember245', 'ember259', 'ember273', 'ember285', 'ember297', 'ember311', 'ember312', 'ember313', 'ember314', 'ember315', 'ember316', 'ember317', 'ember318', 'ember319', 'ember320', 'ember321', 'ember322', 'ember323', 'ember324', 'ember325', 'ember326', 'ember327', 'ember328']


In [25]:
print(driver.current_url)

https://www.linkedin.com/jobs/search/?f_TPR=r2592000&geoId=102454443&keywords=Data%20Scientist&location=Singapore&start=0


### Functions

In [26]:
def page_search(wd, search_location, search_keyword, search_remote, search_posted, search_page, search_count, file, logging):
    # wait time for events in seconds
    page_wait = 30
    click_wait = 5
    async_wait = 5

    # when retrying, number of attempts
    attempts = 3

    # navigate to search page
    #url_search = f"https://www.linkedin.com/jobs/search/?f_TPR={search_posted}&f_WRA={search_remote}&geoId=103644278&keywords={search_keyword}&location={search_location}&start={search_page}"
    url_search = f"https://www.linkedin.com/jobs/search/?f_TPR={search_posted}&geoId={GEO_ID.SG}&keywords={search_keyword}&location={search_location}&start={search_page}"
    wd.get(url_search)
    time.sleep(page_wait) # add sleep so don't get caught

    # find the number of results 
    search_count = wd.find_element_by_xpath("//small[@class='display-flex t-12 t-black--light t-normal']").text
    search_count = int(search_count.split(' ')[0].replace(',', ''))  # get number before space & remove comma (ex. "1,245 results")
    logging.info(f"Loading page {round(search_page/25) + 1} of {round(search_count/25)} for {search_keyword}'s {search_count} results...")

    # get all the job_id's for xpath for current page to click each element
    # running into errors with slow load (11-Aug)
    for attempt in range(attempts):
        try:
            search_results = wd.find_element_by_xpath("//ul[@class='jobs-search-results__list list-style-none']").find_elements_by_tag_name("li")
            result_ids = [result.get_attribute('id') for result in search_results if result.get_attribute('id') != '']
            break
        except:
            time.sleep(click_wait) # wait a few attempts, if not throw an exception and then skip to next page

    # cycle through each job_ids and steal the job data...muhahaha!
    list_jobs = [] #initate a blank list to append each page to
    for id in result_ids:
        try:
            job = wd.find_element_by_id(id) 
            job_id = job.get_attribute("data-occludable-entity-urn").split(":")[-1]
            # select a job and start extracting information
            wd.find_element_by_xpath(f"//div[@data-job-id={job_id}]").click()
        except:
            continue
            # exception likely to job deleteing need to go to next id

        for attempt in range(attempts):
            try:
                # from analysis 3 attempts at 5 second waits gets job titles 99.99% of time (11-Aug)
                job_title = wd.find_element_by_xpath("//h2[@class='t-24 t-bold']") # keep having issues with finding element
                job_title = job_title.text
                break
            except:
                job_title = ''
                time.sleep(click_wait)
        
        # Having issues finding xpath of company (Added 11-Aug)
        for attempt in range(attempts):
            try:
                job_top_card1 = wd.find_element_by_xpath("//span[@class='jobs-unified-top-card__subtitle-primary-grouping mr2 t-black']").find_elements_by_tag_name("span")
                company = job_top_card1[0].text
                location = job_top_card1[1].text
                if len(job_top_card1) > 2: # only displays remote if selected, otherwise only 2 elements in list
                    remote = job_top_card1[2].text
                else:
                    remote = ''
                break
            except:
                company = ''
                location = ''
                remote = ''
                time.sleep(click_wait)
        
        for attempt in range(attempts):
            try:
                #multiple issues with job_top_card2 loading
                job_top_card2 = wd.find_element_by_xpath("//span[@class='jobs-unified-top-card__subtitle-secondary-grouping t-black--light']").find_elements_by_tag_name("span")
                update_time = job_top_card2[0].text
                applicants = job_top_card2[1].text.split(' ')[0]
                break
            except: 
                update_time = '' # after #attempts leave as blank and move on
                applicants = '' # after #attempts leave as blank and move on
                time.sleep(click_wait)

        # Due to (slow) ASYNCHRONOUS updates, need wait times to get job_info
        job_time = '' # assigning as blanks as not important info and can skip if not obtained below
        job_position = ''
        job_pay = ''
        for attempt in range(attempts):
            try:
                # 1 - make sure HTML element is loaded
                element = WebDriverWait(wd, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='mt5 mb2']/div[1]")))
                # 2 - make sure text is loaded
                try:
                    job_info = element.text
                    if job_info != '':
                        # seperate job information on time requirements and position
                        job_info = job_info.split(" · ")
                        if len(job_info) == 1: # only one item means its job _time
                            job_pay = ''
                            job_time = job_info[0]
                            job_position = ''
                        elif (len(job_info) >= 2) and ("$" in job_info[0]): # if has money symbol then seperate
                            job_pay = job_info[0]
                            job_time = job_info[1]
                            if(len(job_info)>= 3): # check if job_info is required
                                job_position = job_info[2]
                            else:
                                job_position = ''
                        else: # else condition satisifies the last condition
                            job_time = job_info[0]
                            job_position = job_info[1]
                            job_pay = ''
                        break
                    else:
                        time.sleep(async_wait)
                except:
                    # error means page didn't load so try again
                    time.sleep(async_wait)
            except:
                # error means page didn't load so try again
                time.sleep(async_wait)

        # get company details and seperate on size and industry
        company_size = '' # assigning as blanks as not important info and can skip if not obtained below
        company_industry = ''
        job_details = ''      
        for attempt in range(attempts):
            try:
                company_details = wd.find_element_by_xpath("//div[@class='mt5 mb2']/div[2]").text
                if " · " in company_details:
                    company_size = company_details.split(" · ")[0]
                    company_industry = company_details.split(" · ")[1]
                else:
                    company_size = company_details
                    company_industry = ''
                job_details = wd.find_element_by_id("job-details").text.replace("\n", " ")
                break
            except: 
                time.sleep(click_wait)

        # append (a) line to file
        date_time = datetime.datetime.now().strftime("%d%b%Y-%H:%M:%S")
        search_keyword = search_keyword.replace("%20", " ")
        list_job = [date_time, search_keyword, search_count, job_id, job_title, company, location, remote, update_time, applicants, job_pay, job_time, job_position, company_size, company_industry, job_details]
        list_jobs.append(list_job)

    with open(file, "a") as f:
        w = csv.writer(f)
        w.writerows(list_jobs)
        list_jobs = []
    
    logging.info(f"Page {round(search_page/25) + 1} of {round(search_count/25)} loaded for {search_keyword}")
    search_page += 25

    return search_page, search_count, url_search

## Main Program

In [27]:

# URL search terms focusing on what type of skills are required for Data Analyst & Data Scientist
search_keywords = ['Data Analyst', 'Data Scientist', 'Data Engineer']


# Counting Exceptions
exception_first = 0
exception_second = 0

for search_keyword in search_keywords:
    search_keyword = search_keyword.lower().replace(" ", "%20")

    # Loop through each page and write results to csv
    search_page = 0 # start on page 1
    search_count = 1 # initiate search count until looks on page
    while (search_page < search_count) and (search_page != 1000):
        # Search each page and return location after each completion
        try:
            search_page, search_count, url_search = page_search(driver, search_location, search_keyword, search_remote, search_posted, search_page, search_count, file, logging)
        except Exception as e:
            logging.error(f'(1) FIRST exception for {search_keyword} on {search_page} of {search_count}, retrying...')
            logging.error(f'Current URL: {url_search}')
            logging.error(e)
            logging.exception('Traceback ->')
            exception_first += 1
            time.sleep(5) 
            try:
                search_page, search_count, url_search = page_search(driver, search_location, search_keyword, search_remote, search_posted, search_page, search_count, file, logging)
                logging.warning(f'Solved Exception for {search_keyword} on {search_page} of {search_count}')
            except Exception as e:
                logging.error(f'(2) SECOND exception remains for {search_keyword}. Skipping to next page...')
                logging.error(f'Current URL: {url_search}')
                logging.error(e)
                logging.exception('Traceback ->')
                search_page += 25 # skip to next page to avoid entry
                exception_second += 1
                logging.error(f'Skipping to next page for {search_keyword}, on {search_page} of {search_count}...')

# close browser
driver.quit()

logging.info(f'LinkedIn data scraping complete with {exception_first} first and {exception_second} second exceptions')
logging.info(f'Regard all further alarms...')

In [28]:
#driver.quit()